In [1]:
import pandas as pd

In [2]:
series = pd.read_csv("./series.csv")
series = series[series["合约系列"] != "小计"]
series["合约系列"] = series["合约系列"].apply(lambda x: x.strip())
series = series.loc[series.groupby("trade_date")["持仓量"].idxmax(), :].set_index(["trade_date", "合约系列"])
# 11.25似乎是au2412集体平仓的日子，当日应开始交易au2502，故以持仓量为筛选标准

In [3]:
contracts = pd.read_csv("./data.csv")
contracts["合约系列"] = contracts["合约代码"].apply(lambda x: x[:6])
contracts["trade_date"] = pd.to_datetime(contracts["trade_date"])
series = series.reset_index()
series["trade_date"] = pd.to_datetime(series["trade_date"])
series = series.set_index(["trade_date", "合约系列"])
series.head(5)

,,成交量,持仓量,持仓量变化,成交额,行权量,隐含波动率
trade_date,合约系列,,,,,,
2024-11-01,au2412,118823,78321,1156,58333.520,0,0.243973
2024-11-04,au2412,92282,78782,461,44226.686,0,0.251494
2024-11-05,au2412,76203,79186,404,33590.252,0,0.247138
2024-11-06,au2412,121009,81294,2108,56180.084,1,0.258216
2024-11-07,au2412,134971,78367,-2927,58951.904,91,0.232901


In [4]:
contracts = pd.read_csv("./data.csv")
contracts["合约系列"] = contracts["合约代码"].apply(lambda x: x[:6])
contracts["trade_date"] = pd.to_datetime(contracts["trade_date"])
series = series.reset_index()
series["trade_date"] = pd.to_datetime(series["trade_date"])
series = series.set_index(["trade_date", "合约系列"])
contracts = contracts.set_index(["trade_date", "合约系列"]).loc[series.index, :].reset_index()
contracts["C/P"] = contracts["合约代码"].apply(lambda x: x[6])
contracts["trade_date"] = pd.to_datetime(contracts["trade_date"])
# 持仓量 or 成交量 or 价格最大/最小 ?
contracts = contracts.loc[contracts.groupby([contracts["trade_date"], contracts["C/P"]])["成交量"].idxmax(), :].set_index(["trade_date", "C/P"])
contracts.head(10)

合约系列                            合约代码    开盘价    最高价   最低价  \
trade_date C/P                                                               
2024-11-01 C    au2412  au2412C640                      11.02  11.02  7.28   
           P    au2412  au2412P600                       3.62   5.10  3.62   
2024-11-04 C    au2412  au2412C640                       9.00  10.48  6.08   
           P    au2412  au2412P600                       4.04   4.70  3.52   
2024-11-05 C    au2412  au2412C640                       7.64   7.64  5.46   
           P    au2412  au2412P600                       4.10   5.00  4.02   
2024-11-06 C    au2412  au2412C640                       7.10   9.20  5.00   
           P    au2412  au2412P600                       4.48   4.92  3.50   
2024-11-07 C    au2412  au2412C640                       4.88   5.12  2.18   
           P    au2412  au2412P600                       6.20   6.44  4.66   

                 收盘价   前结算价    结算价   涨跌1   涨跌2    成交量   持仓量  持仓量变化       成交额  \
trade_date C/P                                                                 
2024-11-01 C    8.90  14.84  11.10 -5.94 -3.74   8214  2152   -219  7367.230   
           P    4.22   3.94   5.28  0.28  1.34  11958  5444    166  5260.700   
2024-11-04 C    6.62  11.10  10.26 -4.48 -0.84   7715  2035   -117  6183.330   
           P    4.30   5.28   5.10 -0.98 -0.18  10193  4926   -518  4333.858   
2024-11-05 C    7.08  10.26   8.46 -3.18 -1.80   7594  2253    218  4974.944   
           P    4.64   5.10   5.38 -0.46  0.28   8732  4787   -139  3888.648   
2024-11-06 C    5.70   8.46   9.62 -2.76  1.16  11380  2499    246  8772.230   
           P    4.56   5.38   5.04 -0.82 -0.34  13148  6043   1256  5584.564   
2024-11-07 C    2.20   9.62   4.24 -7.42 -5.38  10525  3078    579  3076.654   
           P    5.36   5.04   6.92  0.32  1.88  17260  4397  -1646  9467.576   

                     德尔塔  行权量  
trade_date C/P                 
2024-11-01 C    0.402060    0  
           P   -0.221994    0  
2024-11-04 C    0.389270    0  
           P   -0.221815    0  
2024-11-05 C    0.350833    0  
           P   -0.239482    0  
2024-11-06 C    0.377939    0  
           P   -0.224015    0  
2024-11-07 C    0.229025    0  
           P   -0.317808    0

In [5]:
import numpy as np
from scipy.stats import norm
from datetime import datetime

In [6]:
def calculate_time_to_expiry(trade_date, contract_series):
    """
    Calculate time to expiry in years.
    Args:
        trade_date (str or datetime): Trade date.
        contract_series (str): Contract series in the form 'au2412'.
    Returns:
        float: Time to expiry in years.
    """
    # Ensure trade_date is converted to datetime
    if isinstance(trade_date, str):
        trade_date = datetime.strptime(trade_date, "%Y-%m-%d")
    
    # Calculate expiry date based on contract_series
    expiry_date = datetime.strptime(f"20{contract_series[2:4]}-{contract_series[4:6]}-25", "%Y-%m-%d")
    time_to_expiry = (expiry_date - trade_date).days / 365.0
    return max(time_to_expiry, 1 / 365)  # Avoid zero or negative expiry times

def black_scholes_greeks(S, K, T, r, sigma, option_type):
    """
    Calculate Black-Scholes Greeks: delta, gamma, vega, theta, rho, vanna, charm.
    Args:
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiry (in years).
        r (float): Risk-free interest rate.
        sigma (float): Volatility.
        option_type (str): 'C' for call, 'P' for put.
    Returns:
        dict: Dictionary of Greeks.
    """
    # d1 and d2 parameters
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    pdf_d1 = norm.pdf(d1)  # PDF of d1
    cdf_d1 = norm.cdf(d1)  # CDF of d1
    
    # Common factors for Greeks
    discount_factor = np.exp(-r * T)
    sqrt_T = np.sqrt(T)

    # Greeks calculation
    delta = cdf_d1 if option_type == 'C' else cdf_d1 - 1
    gamma = pdf_d1 / (S * sigma * sqrt_T)
    vega = S * pdf_d1 * sqrt_T
    theta = (-S * pdf_d1 * sigma / (2 * sqrt_T)
             - r * K * discount_factor * norm.cdf(d2 if option_type == 'C' else -d2))
    rho = K * T * discount_factor * norm.cdf(d2 if option_type == 'C' else -d2)

    # Additional Greeks
    vanna = -pdf_d1 * d2 / sigma  # Sensitivity of Delta to Volatility
    charm = -pdf_d1 * (1 / (2 * T) + (r - 0.5 * sigma**2) / (sigma * sqrt_T))  # Sensitivity of Delta to Time

    # Return all Greeks as a dictionary
    return {
        'delta': delta,
        'gamma': gamma,
        'vega': vega,
        'theta': theta,
        'rho': rho,
        'vanna': vanna,
        'charm': charm
    }

In [7]:
def calculate_greeks(contracts, series, spot_price_df, risk_free_rate=0.02):
    """
    Main function to calculate Greeks for all contracts using spot prices from spot_price_df.
    Args:
        contracts (DataFrame): Contracts DataFrame.
        series (DataFrame): Series DataFrame with implied volatility.
        spot_price_df (DataFrame): Spot prices DataFrame with trade_date, 合约代码, and spot_price.
        risk_free_rate (float): Risk-free interest rate.
    Returns:
        DataFrame: Updated DataFrame with Greeks.
    """
    # Reset index to access 'trade_date' and '合约系列'
    contracts = contracts.reset_index()
    series = series.reset_index()

    results = contracts.copy()

    # Ensure 'trade_date' in all DataFrames is of datetime type
    results['trade_date'] = pd.to_datetime(results['trade_date'])
    series['trade_date'] = pd.to_datetime(series['trade_date'])
    spot_price_df['trade_date'] = pd.to_datetime(spot_price_df['trade_date'], format='%Y%m%d')

    # Extract strike price and option type (C/P)
    results['strike'] = results['合约代码'].apply(lambda x: ''.join([char for char in x if char.isdigit()])[-3:]).astype(float)
    results['option_type'] = results['合约代码'].str[6]

    # Calculate time to expiry
    results['time_to_expiry'] = results.apply(
        lambda row: calculate_time_to_expiry(row['trade_date'], row['合约系列']), axis=1
    )

    # Merge with series DataFrame to get implied volatility
    results = results.merge(series[['trade_date', '合约系列', '隐含波动率']],
                            on=['trade_date', '合约系列'], how='left')

    # Merge with spot_price DataFrame to get spot price
    results = results.merge(spot_price_df[['trade_date', '合约代码', 'spot_price']],
                            on=['trade_date', '合约代码'], how='left')

    # Check for missing spot prices and fill with default value (optional)
    results['spot_price'] = results['spot_price'].fillna(600)  # Default placeholder spot price

    # Calculate Greeks
    greeks = results.apply(lambda row: black_scholes_greeks(
        S=row['spot_price'],
        K=row['strike'],
        T=row['time_to_expiry'],
        r=risk_free_rate,
        sigma=row['隐含波动率'],
        option_type=row['option_type']
    ), axis=1)

    # Unpack Greeks into separate columns
    greeks_df = pd.DataFrame(greeks.tolist())
    results = pd.concat([results, greeks_df], axis=1)

    # Restore the original index
    results = results.set_index(['trade_date', 'C/P'])

    return results

In [8]:
spot_price = pd.read_csv("./with_spot_price.csv")

In [9]:
updated_contracts = calculate_greeks(contracts, series, spot_price)

In [10]:
updated_contracts

合约系列                            合约代码    开盘价    最高价   最低价  \
trade_date C/P                                                               
2024-11-01 C    au2412  au2412C640                      11.02  11.02  7.28   
           P    au2412  au2412P600                       3.62   5.10  3.62   
2024-11-04 C    au2412  au2412C640                       9.00  10.48  6.08   
           P    au2412  au2412P600                       4.04   4.70  3.52   
2024-11-05 C    au2412  au2412C640                       7.64   7.64  5.46   
           P    au2412  au2412P600                       4.10   5.00  4.02   
2024-11-06 C    au2412  au2412C640                       7.10   9.20  5.00   
           P    au2412  au2412P600                       4.48   4.92  3.50   
2024-11-07 C    au2412  au2412C640                       4.88   5.12  2.18   
           P    au2412  au2412P600                       6.20   6.44  4.66   
2024-11-08 C    au2412  au2412C640                       2.80   3.06  1.76   
           P    au2412  au2412P600                       5.00   5.00  3.04   
2024-11-11 C    au2412  au2412C640                       2.00   2.80  1.26   
           P    au2412  au2412P600                       3.18   3.18  2.36   
2024-11-12 C    au2412  au2412C616                       7.00   7.00  3.06   
           P    au2412  au2412P600                       3.00   5.44  3.00   
2024-11-13 C    au2412  au2412C616                       2.68   3.56  2.38   
           P    au2412  au2412P600                       4.70   5.78  4.20   
2024-11-14 C    au2412  au2412C616                       2.86   3.16  0.92   
           P    au2412  au2412P584                       1.10   3.78  1.10   
2024-11-15 C    au2412  au2412C600                       4.00   5.12  3.28   
           P    au2412  au2412P584                       3.16   3.70  2.04   
2024-11-18 C    au2412  au2412C600                       4.40   7.42  3.10   
           P    au2412  au2412P584                       2.60   2.94  1.26   
2024-11-19 C    au2412  au2412C616                       1.52   2.84  1.32   
           P    au2412  au2412P600                       4.70   5.10  1.86   
2024-11-20 C    au2412  au2412C616                       3.76   4.32  2.02   
           P    au2412  au2412P600                       1.48   2.32  0.82   
2024-11-21 C    au2412  au2412C616                       2.40   5.14  2.24   
           P    au2412  au2412P608                       2.80   2.80  0.90   
2024-11-22 C    au2412  au2412C624                       1.30   3.80  0.84   
           P    au2412  au2412P616                       2.66   3.20  0.48   
2024-11-25 C    au2502  au2502C680                       5.16   5.16  1.60   
           P    au2502  au2502P600                       6.10   8.28  5.78   
2024-11-26 C    au2502  au2502C680                       2.92   3.26  1.82   
           P    au2502  au2502P600                       7.02  11.54  7.02   
2024-11-27 C    au2502  au2502C632                       8.40   9.38  7.40   
           P    au2502  au2502P600                       9.02  10.80  7.80   
2024-11-28 C    au2502  au2502C632                       9.22  10.08  6.34   
           P    au2502  au2502P600                       7.66  10.28  7.56   
2024-11-29 C    au2502  au2502C632                       8.10   9.46  6.92   
           P    au2502  au2502P600                       7.88   9.06  6.70   
2024-12-02 C    au2502  au2502C632                       7.46   9.48  6.28   
           P    au2502  au2502P600                       6.80   8.98  6.38   
2024-12-03 C    au2502  au2502C632                       8.06   9.44  7.28   
           P    au2502  au2502P600                       7.04   7.46  5.92   
2024-12-04 C    au2502  au2502C632                       9.00   9.64  7.84   
           P    au2502  au2502P600                       6.50   6.72  5.72   
2024-12-05 C    au2502  au2502C632                       7.20   9.00  6.98   
           P    au2502  au2502P600      